# Formal Notation



## Definition
A **Neural Network (NN)** is a (of course finite) sequence of layers 
$L^1, L^2,... L^M $ (The superscript serves like an index and does not denote any power-function). If $M > 1$ then
$$
k^{l-1} = d^{l}, \quad \text{for} \quad l \in \{2,..., M\}  
$$


This means, that the output-dimension of Layer $l-1$ is equal to the input-dimension of Layer $l$. So Layer $l$ can use the outgoing activation of Layer $l-1$ as its incoming activation. The first layer will take the feature vector $x$ as incoming activation, compute the outgoing activation and transfer the  the result to the next layer, which uses it as its incoming activation. The outgoing activation of the last layer is the result of the network for input $x$. This process is called _forward propagation_. 

## Input dimension and output dimension of a network

The input dimension of the first layer called **input dimension of the network** of the network. It must be equal to the dimenson of our feature vectors.

The output-dimension of the last layer is called **output dimension of the network**. It must be equal to the dimension of our label-vectors.

## Forward Propagation
Let $\text{NN} = (L^1, L^1,\dots,  L^M)$ be a Neural Network with input dimension $d$, then for  any feature vector $x \in \mathbb{R}^{d}$

$$
NN(x) = (L_M \circ L_{M-1} \circ \ldots \circ  L_1)(x)
$$

The process of calculating $NN(x)$ is called **forward-propagation** of the input $x$.

# LANET 0.2: Neural Network with Forward Propagation

## Notes on the implementation
- The activation of each layer is stored in the layer in the variable `A`.
- To reduce computations we also store the $z$ vector for each layer.
- For simplicity we will add a Layer 0, just to hold the feature vector $x$ in variable `A`. Then we can do forward propagation in a simple loop over all layers
- we return the content of `A` of the last layer as result


## Updated Code

In [0]:
import numpy as np 

def sigma(z, act_func):
    global _activation
    if act_func == 'relu':
       return np.maximum(z, np.zeros(z.shape))
    
    elif act_func == 'sigmoid':
      return 1.0/(1.0 + np.exp( -z ))

    elif act_func == 'linear':
        return z
    else:
        raise Exception('Activation function is not defined.')

class Layer:
    def __init__(self,input_dim, output_dim, activation_function='linear'):    
        self.activation = activation_function
        self.input_dim = input_dim
        self.output_dim = output_dim 
        if input_dim > 0:
            #self.b = np.random.randn( output_dim, 1 )       
            #self.W = np.random.randn( output_dim, input_dim )
            #self.dW = np.random.randn( output_dim, input_dim )
            #self.db = np.random.randn( output_dim, 1 )
            self.b  = np.ones( (output_dim, 1) )       
            self.W  = np.ones( (output_dim, input_dim) )
        self.a = np.zeros( (output_dim,1) )

    
    def set_weight(self, W ):
        self.W = W
      
    def set_bias(self, b ):
        self.b = b
  
    def compute_activation(self, a ): 
        self.z =  np.add( np.dot(self.W, a), self.b)
        self.a =  sigma(self.z, self.activation)
    
    def print( self ):      
        print(f"\n====== Layer Info =======")
        print(f"a    = {self.a}")
        if self.input_dim > 0: 
          print(f"W   =  {self.W}")          
          print(f"b   =  {self.b}")  
    

class Model:
  def __init__(self, input_dim):  
      self.neural_net = []
      self.neural_net.append(Layer(0 , input_dim, 'irrelevant'))    


  def add_layer(self, nr_neurons, activation='relu'):    
      layer_index = len(self.neural_net)
      input_dim = self.neural_net[layer_index - 1].output_dim
      new_layer = Layer( input_dim, nr_neurons, activation)
      self.neural_net.append( new_layer )


  def forward_propagation(self, input_vec ):
      self.neural_net[0].a = input_vec
      for layer_index in range(1,len(self.neural_net)):    
        _A_Prev = self.neural_net[layer_index-1].a                       
        self.neural_net[layer_index].compute_activation( _A_Prev )
        
      return  self.neural_net[layer_index].a
  

  def summary(self):
      print("MODEL SUMMARY")
      for layer_index in range(len(self.neural_net)):        
        self.neural_net[layer_index].print()
        
      print("FINISHED MODEL SUMMARY")

# Test

In [0]:
#Testing        
input_dim = 2
output_dim = 1
model = Model( input_dim )
model.add_layer( 2, 'relu' )
model.add_layer( output_dim, 'linear' )
model.summary()


# Testing for one feature  (N=1)
X  = np.array([[1,1]])
y_predicted = model.forward_propagation( X.T )
print("\nExample 1")
print(f" Shape of X  {X.shape}")
print(f" Result of propagation {y_predicted}")
print(f" Shape of Result of propagation {y_predicted.shape}")

# Testing for more more features (N=2)
print("\nExample 1")
X  = np.array( [[1,1], [2,2]] )
y_predicted = model.forward_propagation( X.T )
print(f" Shape of X  {X.shape}")
print(f" Result of propagation {y_predicted}")
print(f" Shape of Result of propagation {y_predicted.shape}")


MODEL SUMMARY

====== Layer Info =======
a    = [[0.]
 [0.]]

====== Layer Info =======
a    = [[0.]
 [0.]]
W   =  [[1. 1.]
 [1. 1.]]
b   =  [[1.]
 [1.]]

====== Layer Info =======
a    = [[0.]]
W   =  [[1. 1.]]
b   =  [[1.]]
FINISHED MODEL SUMMARY

Example 1
 Shape of X  (1, 2)
 Result of propagation [[7.]]
 Shape of Result of propagation (1, 1)

Example 1
 Shape of X  (2, 2)
 Result of propagation [[ 7. 11.]]
 Shape of Result of propagation (1, 2)
